In [44]:
import pandas as pd
import re

# Đọc dữ liệu từ file CSV vào DataFrame
df = pd.read_csv(r'../crawldata/philong.csv')
df.shape

(474, 379)

In [45]:
df = df.iloc[:, :-359]
df.to_csv(r'../clean data/philongclean.csv', index=False)
df.shape

(474, 20)

In [46]:
print(df.columns)


Index(['Title', 'Price sale', 'Price origin', 'cpu', 'ram', 'ổ cứng',
       'card đồ họa', 'màn hình', 'cổng giao tiếp', 'bàn phím', 'chuẩn lan',
       'chuẩn wifi', 'bluetooth', 'webcam', 'hệ điều hành', 'pin',
       'trọng lượng', 'màu sắc', 'kích thước', 'url'],
      dtype='object')


In [47]:
translation_dict = {
    'Title':'Title', 
    'Price sale':'Price sale',
    'Price origin':'Price origin',
    'cpu': 'CPU',
    'ram': 'RAM',
    'ổ cứng': 'Storage',
    'card đồ họa': 'Graphics card',
    'màn hình': 'Screen',
    'cổng giao tiếp': 'Ports',
    'bàn phím': 'Keyboard',
    'chuẩn lan': 'LAN standard',
    'chuẩn wifi': 'WiFi standard',
    'bluetooth': 'Bluetooth',
    'webcam': 'Webcam',
    'hệ điều hành': 'Operating system',
    'pin': 'Battery',
    'trọng lượng': 'Weight',
    'màu sắc': 'Color',
    'kích thước': 'Dimensions',
    'url': 'URL',
}

df.columns = df.columns.map(translation_dict)


df.to_csv(r'../clean data/philongclean.csv', index=False)

print(df.columns)


Index(['Title', 'Price sale', 'Price origin', 'CPU', 'RAM', 'Storage',
       'Graphics card', 'Screen', 'Ports', 'Keyboard', 'LAN standard',
       'WiFi standard', 'Bluetooth', 'Webcam', 'Operating system', 'Battery',
       'Weight', 'Color', 'Dimensions', 'URL'],
      dtype='object')


In [48]:

split_data = df['Title'].str.split('(')
df['Name'] = split_data.str[0].str.strip()
df['dec'] = split_data.str[1].str.strip()

split_data = df['dec'].str.split('-')
df['CPU'] = split_data.str[0].str.strip()
split_data = df['dec'].str.split(',')
df['RAM'] = split_data.str[1].str.strip()
df['Storage'] = split_data.str[2].str.strip()
df['Screen'] = split_data.str[3].str.strip()
df['Operating_System'] = split_data.str[4].str.strip()

split_data = df['Name'].str.split(' ')
df['Brand'] = split_data.str[1].str.strip()
df['Product'] = split_data.str[2:].str.join(' ').str.strip()

df = df[df['Title'].str.startswith('Laptop')]
df = df.drop(columns='Title')
df = df.drop(columns='Graphics card')
df = df.drop(columns='Keyboard')
df = df.drop(columns='LAN standard')
df = df.drop(columns='Color')
df = df.drop(columns='Ports')
df = df.drop(columns='Bluetooth')
df = df.drop(columns='Webcam')
df = df.drop(columns='WiFi standard')

df.head()



,Price sale,Price origin,CPU,RAM,Storage,Screen,Operating system,Battery,Weight,Dimensions,URL,Name,dec,Operating_System,Brand,Product
0,24.990.000đ,25.990.000đ,Intel Core i7,RAM 16GB,SSD 512GB,RTX 2050 4GB GDDR6,NaN,4Cell 57.5Wh,NaN,NaN,https://philong.com.vn/laptop-acer-nitro-v-anv...,Laptop Acer Nitro V ANV15-51-72VS,"Intel Core i7-13620H, RAM 16GB, SSD 512GB, RTX...",Màn Hình 15.6 inch FHD,Acer,Nitro V ANV15-51-72VS
1,15.990.000đ,20.990.000đ,Intel Core i5,RAM 8GB,SSD 512GB,RTX2050 4GB GDDR6,NaN,NaN,NaN,NaN,https://philong.com.vn/laptop-acer-gaming-aspi...,Laptop Acer Gaming Aspire 5 A515-58GM-53PZ,"Intel Core i5-13420H, RAM 8GB, SSD 512GB, RTX2...",Màn Hình 15.6inch FHD,Acer,Gaming Aspire 5 A515-58GM-53PZ
2,27.190.000đ,29.990.000đ,Core i7,Ram 16GB,SSD 512GB,Màn Hình 14 Inch FHD+ TouchScreen,NaN,NaN,NaN,NaN,https://philong.com.vn/laptop-dell-inspiron-14...,Laptop Dell Inspiron 14 7430 2in1 T7430-i7U165...,"Core i7-1355U, Ram 16GB, SSD 512GB, Màn Hình 1...",Windows 11,Dell,Inspiron 14 7430 2in1 T7430-i7U165W11SLU
4,Liên hệ,NaN,Intel Core I5,RAM 8GB,SSD 512GB,Màn Hình 15.6 inch FHD,NaN,NaN,NaN,NaN,https://philong.com.vn/laptop-dell-vostro-15-3...,Laptop Dell Vostro 15 3520 5M2TT2,"Intel Core I5-1235U, RAM 8GB, SSD 512GB, Màn H...",Windows 11),Dell,Vostro 15 3520 5M2TT2
5,15.490.000đ,16.990.000đ,71030559),NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://philong.com.vn/laptop-dell-vostro-v352...,Laptop Dell Vostro V3520,71030559),NaN,Dell,Vostro V3520


In [49]:
import numpy as np

# Hàm chuyển đổi giá thành số nguyên và xử lý giá trị "Liên hệ"
def convert_price(price):
    if isinstance(price, str):  
        if price == 'Liên hệ':
            return np.nan
        else:
            return int(price.replace('.', '').replace('đ', ''))
    else:
        return price 

# Áp dụng hàm chuyển đổi cho cột 'Price sale' và 'Price origin'
df['Price sale'] = df['Price sale'].apply(convert_price)
df['Price origin'] = df['Price origin'].apply(convert_price)

# Thay thế các giá trị null trong 'Price sale' bằng giá trị trung bình
mean_price_sale = df['Price sale'].mean()
df['Price sale'].fillna(mean_price_sale, inplace=True)

# Thay thế các giá trị null trong 'Price origin' bằng giá trị trong 'Price sale'
df['Price origin'].fillna(df['Price sale'], inplace=True)

df.to_csv(r'../clean data/philongclean.csv', index=False)
df.head()


C:\Users\OS\AppData\Local\Temp\ipykernel_16580\3769437774.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Price sale'].fillna(mean_price_sale, inplace=True)
C:\Users\OS\AppData\Local\Temp\ipykernel_16580\3769437774.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

,Price sale,Price origin,CPU,RAM,Storage,Screen,Operating system,Battery,Weight,Dimensions,URL,Name,dec,Operating_System,Brand,Product
0,2.499000e+07,2.599000e+07,Intel Core i7,RAM 16GB,SSD 512GB,RTX 2050 4GB GDDR6,NaN,4Cell 57.5Wh,NaN,NaN,https://philong.com.vn/laptop-acer-nitro-v-anv...,Laptop Acer Nitro V ANV15-51-72VS,"Intel Core i7-13620H, RAM 16GB, SSD 512GB, RTX...",Màn Hình 15.6 inch FHD,Acer,Nitro V ANV15-51-72VS
1,1.599000e+07,2.099000e+07,Intel Core i5,RAM 8GB,SSD 512GB,RTX2050 4GB GDDR6,NaN,NaN,NaN,NaN,https://philong.com.vn/laptop-acer-gaming-aspi...,Laptop Acer Gaming Aspire 5 A515-58GM-53PZ,"Intel Core i5-13420H, RAM 8GB, SSD 512GB, RTX2...",Màn Hình 15.6inch FHD,Acer,Gaming Aspire 5 A515-58GM-53PZ
2,2.719000e+07,2.999000e+07,Core i7,Ram 16GB,SSD 512GB,Màn Hình 14 Inch FHD+ TouchScreen,NaN,NaN,NaN,NaN,https://philong.com.vn/laptop-dell-inspiron-14...,Laptop Dell Inspiron 14 7430 2in1 T7430-i7U165...,"Core i7-1355U, Ram 16GB, SSD 512GB, Màn Hình 1...",Windows 11,Dell,Inspiron 14 7430 2in1 T7430-i7U165W11SLU
4,2.722989e+07,2.722989e+07,Intel Core I5,RAM 8GB,SSD 512GB,Màn Hình 15.6 inch FHD,NaN,NaN,NaN,NaN,https://philong.com.vn/laptop-dell-vostro-15-3...,Laptop Dell Vostro 15 3520 5M2TT2,"Intel Core I5-1235U, RAM 8GB, SSD 512GB, Màn H...",Windows 11),Dell,Vostro 15 3520 5M2TT2
5,1.549000e+07,1.699000e+07,71030559),NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://philong.com.vn/laptop-dell-vostro-v352...,Laptop Dell Vostro V3520,71030559),NaN,Dell,Vostro V3520


In [50]:

df = df.drop(columns='Battery')
df = df.drop(columns='Weight')
df = df.drop(columns='Dimensions')
df = df.drop(columns='Operating_System')

df.rename(columns={'RAM': 'Ram'}, inplace=True)

df.rename(columns={'Name': 'Title'}, inplace=True)
df.rename(columns={'Product': 'Name'}, inplace=True)

df.to_csv(r'../clean data/philongclean.csv', index=False)



In [51]:
def update_os(row):
    if row['Brand'] == 'Mac':
        return 'Mac'
    else:
        return 'Windows 11'
df['Operating system'] = df.apply(update_os, axis=1)

pattern = r'\b(\d*\.?\d+)\s?(?:inch|Inch|INCH)\b'
def extract_screen_size(dec):
    match = re.search(pattern, dec)
    if match:
        return match.group(1)
    else:
        return None
df['Screen'] = df['dec'].apply(extract_screen_size)

pattern = r'(\d+\.?\d*)\s?(GB|T)'
def extract_ram(dec):
    match = re.search(pattern, dec)
    if match:
        size = float(match.group(1))
        unit = match.group(2)
        if unit == 'T':
            return  1024 
        else:
            return str(size)
    else:
        return None
df['Ram'] = df['dec'].apply(extract_ram)

pattern = r'SSD\s+(\d+\.?\d*)\s?(GB|TB)'
def extract_storage(dec):
    match = re.search(pattern, dec)
    if match:
        size = float(match.group(1))
        unit = match.group(2)
        if unit == 'TB':
            return size*1024
        else:
            return str(size)
    else:
        return None

df['Storage'] = df['dec'].apply(extract_storage)



In [52]:

df.rename(columns={'Screen': 'Screen size'}, inplace=True)
df['Screen size'].fillna('15', inplace=True)
df['Ram'].fillna('8', inplace=True)
df['Storage'].fillna('256', inplace=True)

df.to_csv(r'../clean data/philongclean.csv', index=False)


C:\Users\OS\AppData\Local\Temp\ipykernel_16580\4065512741.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Screen size'].fillna('15', inplace=True)
C:\Users\OS\AppData\Local\Temp\ipykernel_16580\4065512741.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when

In [54]:
cpu_pattern = r'(?i)i\d+'
def extract_cpu_type(text):
    match = re.search(cpu_pattern, str(text))
    if match:
        return match.group(0)
    return text
df['CPU'] = df['CPU'].apply(extract_cpu_type)


cpu_pattern = r'(?i)i\d+|Ryzen\s+\d+|Core\s+Ultra\s+\d+'

def extract_cpu_type1(text):
    match = re.search(cpu_pattern, str(text))
    if match:
        return match.group(0)
    return text


# Áp dụng hàm rút trích thông tin CPU cho cột CPU type và lưu vào cột mới 'CPU'
df['CPU'] = df['CPU'].apply(extract_cpu_type1)

df.to_csv('../clean data/philongclean.csv', index=False)